<a href="https://colab.research.google.com/github/chodongsu/ChoSanghyuk.github.io/blob/master/daum_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!git clone https://github.com/ChoSanghyuk/EncoreProject.git

Cloning into 'EncoreProject'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 39 (delta 14), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (39/39), done.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup as bs

import pandas as pd
import time

In [ ]:
# 파싱하여 각 뉴스의 기존 url 수집
url = 'https://search.daum.net/search?w=news&cluster=n&q=%EA%B3%B5%EA%B3%B5%EC%9D%98%EB%8C%80&sort=recency&DA=23A'
web = requests.get(url).content
source = bs(web,"html.parser")

In [ ]:
new_url_content = [] # 페이지에 있는 모든 뉴스 url 리스트
urls_list = []       # 다음뉴스의 url의 리스트

for urls in source.find_all('a',{'class' : 'f_link_b'}):
    new_url_content.append(urls['href']) 
for i in new_url_content:     # 무조건 다음뉴스의 url은 f=o으로 끝남
    if i[-3:] == "f=o":
      new_url_content_1 = i[-21:-4]  # 기사 업로드 시간만 슬라이싱
      new_url = "http://news.v.daum.net/v/" + new_url_content_1  # daum 뉴스 url 형식에 기사 업로드 시간 융합

      urls_list.append(new_url) 

urls_list

['http://news.v.daum.net/v/20200919225101729',
 'http://news.v.daum.net/v/20200919203408561',
 'http://news.v.daum.net/v/20200919122947872',
 'http://news.v.daum.net/v/20200919113404284',
 'http://news.v.daum.net/v/20200919111746109',
 'http://news.v.daum.net/v/20200919101602311',
 'http://news.v.daum.net/v/20200919095601005']

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/chromedriver.exe"

'/content/drive/My Drive/Colab Notebooks/chromedriver.exe'


In [ ]:
driver = webdriver.Chrome(executable_path='/content/drive/My Drive/Colab Notebooks/chromedriver.exe')

WebDriverException: ignored

In [ ]:
contents_list = []  # 수집한 댓글 보관 리스트

for i in range(10): # 10개의 url 수집
  driver.get(urls_list[i])
  print(driver.current_url)

  time.sleep(3)
  try:
    # 더보기 버튼이 없을 때까지 클릭
    while driver.find_element_by_xpath('//*[@id="alex-area"]/div/div/div/div[3]/div[2]/a').text !='':   
       driver.find_element_by_xpath('//*[@id="alex-area"]/div/div/div/div[3]/div[2]/a').click()         
       time.sleep(3)
    except:
      pass

    time.sleep(2)

    # 최대 댓글 수집 개수 조절
    for k in range(40):
      try:
        # 좋아요와 싫어요 수에 대한 x_paht 위치
        like_num = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/ul[2]/li[" +str(k)+ "]/div/div/span[2]/button[1]/span[2]").text
        unlike_num = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/ul[2]/li[" +str(k)+ "]/div/div/span[2]/button[2]/span[2]").text

        # x_path를 이용해 댓글 추출(좋아요 수가 10개가 넘을 때)
        if int(like_num) >= 10:
          contents = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/ul[2]/li[' +str(k)+']/div/p').text
          
        # 줄 바꿈 무시
        contents = contents.replace('\n'," ")
        contents_list.append(contents)

      except:
        pass
      
      time.sleep(3)
